# Dados

In [1]:
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import matplotlib.animation as animation
import numpy as np

In [2]:
boston=pd.read_csv('dados/boston.csv',header=None)
boston=boston.rename(columns={boston.columns[0]:'x',boston.columns[1]:'z',boston.columns[2]:'w',boston.columns[3]:'h',boston.columns[4]:'i',
    boston.columns[5]:'e',boston.columns[6]:'o',boston.columns[7]:'l',boston.columns[8]:'g',boston.columns[9]:'f',boston.columns[10]:'s',boston.columns[11]:'u',
    boston.columns[12]:'k',boston.columns[13]:'y'})
boston

,x,z,w,h,i,e,o,l,g,f,s,u,k,y
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [3]:
gauss=pd.read_csv('dados/gauss.csv')
gauss

,x,y
0,1.0,97.587760
1,2.0,97.763440
2,3.0,96.567050
3,4.0,92.520370
4,5.0,91.150970
...,...,...
245,246.0,8.362109
246,247.0,8.983507
247,248.0,3.362469
248,249.0,1.182678


# Modelo

In [105]:
class PolinomialRegression:
  def __init__(self):
    self.w =[]
    self.n = 0

  def fit(self, x, y,n=1,l=0.1):
    x_poli = self.polinomial_transfomation(x,n)
    self.fit_OLS(x_poli,y,l)
    
  def polinomial_transfomation(self,x,n):
    df_columns = x.columns
    poli_columns={}
    self.n=n
    x_poli=x.copy()
    for column in df_columns:
      for i in range(2,n+1):
        poli_columns[f"{column}_{i}"] = x[column]**i
    poli_df = pd.DataFrame(poli_columns)
    x_poli = pd.concat([x,poli_df], axis=1)
    return x_poli

  def fit_OLS(self, X, y,l):
     X = X.to_numpy()  #Converte X para um array numpy
     X_b = np.c_[np.ones((X.shape[0], 1)), X]  #Adiciona a coluna de bias
     y = y.to_numpy()  #Converte y em um array numpy
     dimensoes= X_b.shape[1]
     I=np.eye(dimensoes)
     w = np.linalg.pinv((X_b.T @ X_b)+(l * I)) @ X_b.T @ y
     self.w=w  #Salva o vetor de Ws
     self.w = self.w.ravel()

  def predict(self, X):
    y_pred=[]# Lista para salvar as predições
    df_bias = pd.DataFrame({'bias':[1]*len(X)})
    x_poli = self.polinomial_transfomation(X,self.n)
    X_bias=pd.concat([df_bias,x_poli], axis=1)
    for sample in range(len(X_bias)):
      y_pred.append((X_bias.iloc[sample]*self.w).sum())# Calcula a predição e salva na lista
    return y_pred

# Normalizador

In [88]:
class Nomalizer:
    def __init__(self):
        self.y_max=0.0
        self.y_min=0.0
        self.x_max=[]
        self.x_min=[]

    def fit(self,x,y):
        # Pega os valores minimos de 
        self.y_max=y['y'].max().tolist()
        self.y_min=y['y'].min().tolist()

        # Pega os valores maximos e minimos do X
        self.x_max=x.max().tolist()
        self.x_min=x.min().tolist()
    
    def normalize(self,x,y):
        y_norm=y.map(self.y_norm_aux)
        x_norm=x.copy()
        norm_index=0
        for column in x.columns:
            x_column=[]
            for i in range(len(x[column])):
                x_column.append((x[column][i]-self.x_min[norm_index])/(self.x_max[norm_index]-self.x_min[norm_index]))
                
            norm_index+=1
            x_norm[column]=x_column    
        
        return x_norm,y_norm
    
    def desnormalize(self,x_norm,y_norm):
        y=y_norm.map(self.y_desnorm_aux)
        x=x_norm.copy()
        norm_index=0
        for column in x_norm.columns:
            x_column=[]
            for i in range(len(x_norm[column])):
                x_column.append((x_norm[column][i]*(self.x_max[norm_index]-self.x_min[norm_index]))+self.x_min[norm_index])
                
            norm_index+=1
            x[column]=x_column    
        
        return x,y
    
    def y_norm_aux(self,y):
        return (y-self.y_min)/(self.y_max-self.y_min)
    
    def y_desnorm_aux(self,y):
        return (y*(self.y_max-self.y_min))+self.y_min


# Questão 1

In [ ]:
# Separação dos dados
boston_train,boston_test = train_test_split(boston, test_size=0.2, random_state=42)
boston_train=boston_train.reset_index(drop=True)
boston_test=boston_test.reset_index(drop=True)

In [ ]:
# Formaçao de x e y do train test 
train_x=boston_train[['x','z','w','h','i','e','o','l','g','f','s','u','k']]
train_y=boston_train[['y']]

test_x=boston_test[['x','z','w','h','i','e','o','l','g','f','s','u','k']]
test_y=boston_test[['y']]

In [89]:
norm=Nomalizer()

In [ ]:
# Ajuste do normalizador
norm.fit(train_x,train_y)

In [ ]:
# Normalização do train e test
norm_Trx,norm_Try=norm.normalize(train_x,train_y)
norm_Tex,norm_Tey=norm.normalize(test_x,test_y)

In [106]:
# Definição e treino do modelo
poli=PolinomialRegression()
poli.fit(norm_Trx,norm_Try,3)

In [ ]:
# Teste da predição
ypred_norm=poli.predict(norm_Tex)
ypred_norm_df=pd.DataFrame(ypred_norm)
x_test,y_pred=norm.desnormalize(norm_Tex,ypred_norm_df)
y_pred

,0
0,27.807051
1,34.625057
2,16.233753
3,25.742325
4,16.601582
...,...
97,9.749873
98,11.471185
99,14.934238
100,20.841850


In [100]:
test_y

,y
0,23.6
1,32.4
2,13.6
3,22.8
4,16.1
...,...
97,17.9
98,9.6
99,17.2
100,22.5
